# ETL Pipeline Preparation
Follow the instructions below to help you create your ETL pipeline.
### 1. Import libraries and load datasets.
- Import Python libraries
- Load `messages.csv` into a dataframe and inspect the first few lines.
- Load `categories.csv` into a dataframe and inspect the first few lines.

In [1]:
# import libraries
import pandas as pd
from time import time
from sqlalchemy import create_engine
from sqlalchemy.pool import NullPool
#import re
#import udacourse #my library!
import math

In [2]:
gen_begin = time()

# load messages dataset
messages = pd.read_csv('messages.csv', index_col='id')
#messages['id'] = messages.index
#messages.info()
#messages.index
messages.head()

,message,original,genre
id,,,
2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [3]:
# load categories dataset
categories = pd.read_csv('categories.csv', index_col='id')
#categories.info()
categories.head()

,categories
id,
2,related-1;request-0;offer-0;aid_related-0;medi...
7,related-1;request-0;offer-0;aid_related-1;medi...
8,related-1;request-0;offer-0;aid_related-0;medi...
9,related-1;request-1;offer-0;aid_related-1;medi...
12,related-1;request-0;offer-0;aid_related-0;medi...


### 6. Remove duplicates.
- Check how many duplicates are in this dataset.
- Drop the duplicates.
- Confirm duplicates were removed
- Index duplicated [here](https://stackoverflow.com/questions/35084071/concat-dataframe-reindexing-only-valid-with-uniquely-valued-index-objects)

In [4]:
#messages.index.duplicated(keep='first')
#drop index duplicated messages
messages = messages.loc[~messages.index.duplicated(keep='first')]

In [5]:
#check number of remaining duplicated messages
print(messages[messages.duplicated()].shape[0])
#drop duplicates
messages = messages.drop_duplicates()
print(messages.shape[0])
# check number of duplicates
messages[messages.duplicated()].shape[0]

3
26177


0

### 2. Merge datasets.
- Merge the messages and categories datasets using the common id
- Assign this combined dataset to `df`, which will be cleaned in the following steps

- used SQL-type of relations, having the messages dataframe as refference
- documentation [here](https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html)

In [6]:
# merge datasets
df = pd.merge(messages, categories, left_index=True, right_index=True, how='left')
print(df.shape[0])
df.head()

26245


,message,original,genre,categories
id,,,,
2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,related-1;request-0;offer-0;aid_related-0;medi...
7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,related-1;request-0;offer-0;aid_related-1;medi...
8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,related-1;request-0;offer-0;aid_related-0;medi...
9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,related-1;request-1;offer-0;aid_related-1;medi...
12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,related-1;request-0;offer-0;aid_related-0;medi...


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26245 entries, 2 to 30265
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   message     26245 non-null  object
 1   original    10184 non-null  object
 2   genre       26245 non-null  object
 3   categories  26245 non-null  object
dtypes: object(4)
memory usage: 1.0+ MB


### 3. Split `categories` into separate category columns.
- Split the values in the `categories` column on the `;` character so that each value becomes a separate column. You'll find [this method](https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.Series.str.split.html) very helpful! Make sure to set `expand=True`.
- Use the first row of categories dataframe to create column names for the categories data.
- Rename columns of `categories` with new column names.

Solution [here](https://stackoverflow.com/questions/42049147/convert-list-to-pandas-dataframe-column)

In [8]:
# create a dataframe of the 36 individual category columns
jumber = df['categories'].iloc[0]
categories = jumber.split(sep=';')
#{'categories': categories}
categories = pd.DataFrame({'categories': categories})
categories.head()

,categories
0,related-1
1,request-0
2,offer-0
3,aid_related-0
4,medical_help-0


In [9]:
# select the first row of the categories dataframe
row = categories.iloc[0]
row['categories'][:-2]

[cat[:-2] for cat in categories['categories']]

# use this row to extract a list of new column names for categories.
# one way is to apply a lambda function that takes everything 
# up to the second to last character of each string with slicing
category_colnames = [cat[:-2] for cat in categories['categories']]
print(category_colnames)

# rename the columns of `categories`
#categories.columns = category_colnames
#categories.head()

['related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 'search_and_rescue', 'security', 'military', 'child_alone', 'water', 'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport', 'buildings', 'electricity', 'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report']


An alert flag, if there was no category inserted:

In [10]:
df['if_blank'] = False

In [11]:
#adding new columns with zero value
for colname in category_colnames:
    df[colname] = 0

#df.columns
print('new shape:',df.shape[1])
#df.head(1)

new shape: 41


In [12]:
cell = df['categories'].iloc[0]
alfa = set(cell.split(sep=';'))
alfa

for beta in alfa:
    if beta.find('1') != -1:
        print(beta[:-2])

related


### 4. Convert category values to just numbers 0 or 1.
- Iterate through the category columns in df to keep only the last character of each string (the 1 or 0). For example, `related-0` becomes `0`, `related-1` becomes `1`. Convert the string to a numeric value.
- You can perform [normal string actions on Pandas Series](https://pandas.pydata.org/pandas-docs/stable/text.html#indexing-with-str), like indexing, by including `.str` after the Series. You may need to first convert the Series to be of type string, which you can do with `astype(str)`.

In [13]:
begin = time()

filtered_cols = ['categories', 'if_blank', 'related', 'request', 'offer', 'aid_related', 'medical_help', 
       'medical_products', 'search_and_rescue', 'security', 'military', 'child_alone', 'water', 'food', 'shelter', 
       'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport', 
       'buildings', 'electricity', 'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report']

def fn_test(x, verbose=False):
    if verbose:
        print('###')      
    string_cat = x['categories']
    #at least one case found
    if string_cat.find('1') != -1:
        #break into components
        alfa = set(string_cat.split(sep=';'))
        #loop on components
        for beta in alfa:
            if beta.find('1') != -1:
                if verbose:
                    print(beta[:-2])
                gama = beta[:-2]
                x[gama] = 1
    #no cases!
    else:
        if verbose:
            print('*empty element*')
        x['if_blank'] = True
        
    return x

df[filtered_cols] = df[filtered_cols].apply(lambda x: fn_test(x, verbose=True), axis=1)

spent = time() - begin
print('elapsed time: {:.1f}s ({}min, {:.4f}sec)'.format(spent, math.trunc((spent)/60), math.fmod(spent, 60)))
df.head()

###
related
###
related
other_aid
storm
weather_related
aid_related
###
related
###
request
hospitals
infrastructure_related
related
buildings
other_aid
aid_related
medical_products
###
related
###
*empty element*
###
related
storm
weather_related
###
request
related
direct_report
shelter
water
aid_related
###
*empty element*
###
request
related
direct_report
medical_help
aid_related
medical_products
###
food
request
transport
infrastructure_related
related
buildings
other_aid
other_infrastructure
floods
direct_report
weather_related
medical_help
water
aid_related
medical_products
###
*empty element*
###
request
related
direct_report
water
aid_related
###
food
request
related
direct_report
aid_related
###
related
###
food
request
related
direct_report
clothing
shelter
water
aid_related
medical_products
###
food
request
related
direct_report
medical_help
water
aid_related
###
*empty element*
###
*empty element*
###
related
weather_related
earthquake
###
related
###
request
related
other

*empty element*
###
request
related
direct_report
clothing
aid_related
medical_products
###
related
###
*empty element*
###
food
request
hospitals
infrastructure_related
related
direct_report
medical_help
water
aid_related
medical_products
###
related
###
request
related
buildings
other_aid
direct_report
aid_related
###
related
###
request
related
other_aid
direct_report
aid_related
###
food
request
related
other_aid
direct_report
clothing
medical_help
shelter
water
aid_related
###
food
request
related
direct_report
medical_help
water
aid_related
medical_products
###
food
request
related
money
direct_report
shelter
water
aid_related
medical_products
###
request
related
clothing
aid_related
###
food
request
related
money
direct_report
medical_help
shelter
aid_related
medical_products
###
request
related
other_aid
direct_report
aid_related
###
refugees
request
related
other_aid
money
direct_report
aid_related
###
related
###
food
request
related
direct_report
medical_help
water
aid_relat

related
direct_report
aid_related
###
refugees
hospitals
infrastructure_related
related
security
weather_related
aid_related
cold
###
*empty element*
###
*empty element*
###
related
###
*empty element*
###
refugees
related
other_aid
aid_related
###
*empty element*
###
*empty element*
###
*empty element*
###
request
related
direct_report
###
*empty element*
###
*empty element*
###
*empty element*
###
related
###
food
request
related
direct_report
aid_related
###
request
transport
related
other_aid
floods
direct_report
weather_related
medical_help
aid_related
###
food
request
related
buildings
direct_report
medical_help
water
aid_related
###
*empty element*
###
related
###
*empty element*
###
request
related
other_aid
direct_report
aid_related
###
food
request
related
direct_report
clothing
death
medical_help
shelter
water
aid_related
medical_products
###
related
other_aid
aid_related
###
*empty element*
###
request
related
direct_report
shelter
aid_related
###
*empty element*
###
food
r

food
request
related
direct_report
water
aid_related
###
food
request
related
direct_report
aid_related
###
food
request
transport
related
floods
direct_report
weather_related
medical_help
shelter
aid_related
###
food
request
infrastructure_related
related
other_infrastructure
shelter
aid_related
###
request
related
other_aid
direct_report
aid_related
###
*empty element*
###
food
request
related
aid_related
###
request
search_and_rescue
related
direct_report
medical_help
aid_related
###
food
request
hospitals
infrastructure_related
related
direct_report
medical_help
water
aid_related
###
food
request
related
aid_related
###
request
related
direct_report
medical_help
aid_related
medical_products
###
food
request
related
medical_help
shelter
water
aid_related
###
request
related
direct_report
###
food
request
related
buildings
other_aid
money
direct_report
shelter
water
aid_related
###
food
request
related
direct_report
water
aid_related
###
food
request
related
direct_report
water
aid_r

###
related
###
food
request
related
direct_report
weather_related
earthquake
aid_related
###
request
related
other_aid
direct_report
shelter
aid_related
###
request
infrastructure_related
related
buildings
other_aid
other_infrastructure
direct_report
shelter
aid_related
###
food
request
related
direct_report
water
aid_related
medical_products
###
food
request
related
direct_report
shelter
water
aid_related
###
related
###
related
###
related
###
request
related
direct_report
water
aid_related
###
related
###
request
related
water
aid_related
###
related
###
food
request
infrastructure_related
related
other_infrastructure
direct_report
death
aid_related
###
refugees
request
related
buildings
direct_report
death
shelter
aid_related
###
food
request
infrastructure_related
related
other_infrastructure
direct_report
aid_related
###
request
related
other_aid
direct_report
shelter
aid_related
###
related
###
food
request
search_and_rescue
infrastructure_related
aid_centers
related
other_weat

...

###
related
###
request
related
direct_report
###
food
request
related
money
direct_report
aid_related
###
food
related
water
aid_related
###
related
###
related
other_weather
buildings
weather_related
electricity
###
*empty element*
###
related
other_weather
weather_related
water
aid_related
###
related
###
request
related
security
direct_report
aid_related
military
###
*empty element*
###
*empty element*
###
related
other_aid
aid_related
###
related
fire
weather_related
water
aid_related
###
transport
related
electricity
###
infrastructure_related
related
other_infrastructure
storm
weather_related
###
food
related
other_weather
storm
weather_related
aid_related
###
transport
related
buildings
direct_report
storm
weather_related
electricity
###
related
###
*empty element*
###
related
medical_help
aid_related
###
*empty element*
###
*empty element*
###
food
transport
related
floods
direct_report
weather_related
clothing
medical_help
shelter
aid_related
###
related
###
*empty element*
#

floods
weather_related
###
transport
related
other_aid
aid_related
###
request
infrastructure_related
aid_centers
related
other_weather
weather_related
aid_related
military
###
*empty element*
###
transport
related
other_aid
aid_related
###
*empty element*
###
search_and_rescue
related
direct_report
water
aid_related
medical_products
###
request
transport
related
floods
direct_report
weather_related
medical_help
water
aid_related
medical_products
###
*empty element*
###
related
shelter
water
aid_related
###
related
death
aid_related
###
request
related
medical_help
aid_related
medical_products
###
*empty element*
###
infrastructure_related
related
other_infrastructure
###
related
###
search_and_rescue
related
floods
weather_related
aid_related
###
refugees
related
aid_related
military
###
related
floods
direct_report
weather_related
shelter
aid_related
###
food
related
aid_related
###
related
other_weather
buildings
weather_related
aid_related
medical_products
###
related
other_weather

*empty element*
###
related
###
*empty element*
###
food
request
shops
infrastructure_related
related
tools
other_aid
direct_report
weather_related
earthquake
aid_related
medical_products
###
related
buildings
shelter
aid_related
###
related
storm
weather_related
shelter
aid_related
###
request
transport
related
floods
direct_report
weather_related
medical_help
water
aid_related
###
*empty element*
###
*empty element*
###
related
security
direct_report
aid_related
###
related
###
*empty element*
###
related
shelter
aid_related
medical_products
###
related
###
*empty element*
###
refugees
related
shelter
aid_related
###
related
###
*empty element*
###
related
other_aid
medical_help
shelter
water
aid_related
medical_products
###
offer
related
medical_help
aid_related
###
*empty element*
###
food
related
medical_help
shelter
aid_related
medical_products
###
*empty element*
###
request
related
direct_report
###
*empty element*
###
related
death
medical_help
aid_related
###
related
###
infr

###
related
elapsed time: 36.2s (0min, 36.1800sec)


,message,original,genre,categories,if_blank,related,request,offer,aid_related,medical_help,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
id,,,,,,,,,,,,,,,,,,,,,
2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,related-1;request-0;offer-0;aid_related-0;medi...,False,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,related-1;request-0;offer-0;aid_related-1;medi...,False,1,0,0,1,0,...,0,0,1,0,1,0,0,0,0,0
8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,related-1;request-0;offer-0;aid_related-0;medi...,False,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,related-1;request-1;offer-0;aid_related-1;medi...,False,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,related-1;request-0;offer-0;aid_related-0;medi...,False,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 5. Replace `categories` column in `df` with new category columns.
- Drop the categories column from the df dataframe since it is no longer needed.
- Concatenate df and categories data frames.

In [14]:
df = df.drop('categories', axis=1)
df.head(1)

,message,original,genre,if_blank,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
id,,,,,,,,,,,,,,,,,,,,,
2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,False,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 7. Save the clean dataset into an sqlite database.
You can do this with pandas [`to_sql` method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html) combined with the SQLAlchemy library. Remember to import SQLAlchemy's `create_engine` in the first cell of this notebook to use it below.

drop table function [here](import logging
from sqlalchemy import MetaData
from sqlalchemy import create_engine
from sqlalchemy.engine.url import URL
from sqlalchemy.ext.declarative import declarative_base)

Better way to do this, [here](https://stackoverflow.com/questions/8645250/how-to-close-sqlalchemy-connection-in-mysql)

- according to the answerer, "Engine is a **factory** for connections as well as a pool of connections, not the connection itself";
- so, to avoid the problem of couldn´t closse the connection, as other members of the poll remaining asking for transactions, the answerer recommends to use **poolclass=NulPool**;
- as we are not dealing with something that really needs a pool (only one transaction per time for us is enough), let´s do it!

In [15]:
database = create_engine('sqlite:///Messages.db', poolclass=NullPool) #, echo=True)
connection = database.connect()

#attempt to save my dataframe to SQLite
try:
    df.to_sql('Messages', database, index=False, if_exists='replace')
except ValueError:
    print('something went wrong when was writing data do SQLite')
    
connection.close()

In [16]:
spent = time() - gen_begin
print('total elapsed time: {:.1f}s ({}min, {:.4f}sec)'.format(spent, math.trunc((spent)/60), math.fmod(spent, 60)))
df.head()

total elapsed time: 39.8s (0min, 39.7848sec)


,message,original,genre,if_blank,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
id,,,,,,,,,,,,,,,,,,,,,
2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,False,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,False,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,False,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,False,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,False,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 8. Use this notebook to complete `etl_pipeline.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database based on new datasets specified by the user. Alternatively, you can complete `etl_pipeline.py` in the classroom on the `Project Workspace IDE` coming later.

In [17]:
def fn_count_valids(dataset, field, criteria=1, verbose=False):
    if verbose:
        print('###counting function initiated')
    
    return dataset[field][dataset[field] == criteria].sum()

In [18]:
df.shape[0]

26245

In [19]:
total = df.shape[0]
field = 'if_blank'
count = fn_count_valids(dataset=df, field=field, criteria=True)
percent = 100. * (count / total)
print('{}:{} ({:.1f}%)'.format(field, count, percent))

if_blank:6317 (24.1%)


In [20]:
expand_list = ['related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 'search_and_rescue', 
       'security', 'military', 'child_alone', 'water', 'food', 'shelter', 'clothing', 'money', 'missing_people', 
       'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport', 'buildings', 'electricity', 'tools', 
       'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 'weather_related', 'floods', 'storm', 'fire', 
       'earthquake', 'cold', 'other_weather', 'direct_report']

verbose = False
total = df.shape[0]
counts = []

for field in expand_list:
    count = fn_count_valids(dataset=df, field=field)
    percent = 100. * (count / total)
    counts.append((count, field, percent))
    if verbose:
        print('{}:{} ({:.1f}%)'.format(field, count, percent))

Order Tupple [here](https://www.pythoncentral.io/how-to-sort-a-list-tuple-or-object-with-sorted-in-python/)

In [21]:
def fn_getKey(item):
    return item[0]

In [22]:
sorted_tuples = sorted(counts, key=fn_getKey, reverse=True)

i=1

for cat in sorted_tuples:
    count, field, percent = cat
    print('{}-{}:{} ({:.1f}%)'.format(i, field, count, percent))
    i += 1

1-related:19928 (75.9%)
2-aid_related:10877 (41.4%)
3-weather_related:7304 (27.8%)
4-direct_report:5080 (19.4%)
5-request:4480 (17.1%)
6-other_aid:3448 (13.1%)
7-food:2930 (11.2%)
8-earthquake:2455 (9.4%)
9-storm:2448 (9.3%)
10-shelter:2319 (8.8%)
11-floods:2158 (8.2%)
12-medical_help:2086 (7.9%)
13-infrastructure_related:1705 (6.5%)
14-water:1674 (6.4%)
15-other_weather:1376 (5.2%)
16-buildings:1335 (5.1%)
17-medical_products:1313 (5.0%)
18-transport:1203 (4.6%)
19-death:1196 (4.6%)
20-other_infrastructure:1151 (4.4%)
21-refugees:876 (3.3%)
22-military:860 (3.3%)
23-search_and_rescue:724 (2.8%)
24-money:604 (2.3%)
25-electricity:534 (2.0%)
26-cold:530 (2.0%)
27-security:471 (1.8%)
28-clothing:406 (1.5%)
29-aid_centers:309 (1.2%)
30-missing_people:299 (1.1%)
31-hospitals:283 (1.1%)
32-fire:282 (1.1%)
33-tools:159 (0.6%)
34-shops:120 (0.5%)
35-offer:118 (0.4%)
36-child_alone:0 (0.0%)


---

## Test area (can be removed later)

In [23]:
raise Exception('Test area')

Exception: Test area

In [ ]:
full_path = 'c://host/pyprog/project/udacourse/categories.csv'
full_path = 'c://host/pyprog/project/udacourse/messages.csv'
last_one = full_path.rfind('/')
full_path[last_one+1:-4]

In [ ]:
categories.index.name = 'categories'
categories.index.name